In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import joblib

# Load the data
train = pd.read_csv('train_FD004.csv', low_memory=False)

# Define a function to calculate Fisher score using first 50 and last 50 samples of each engine
def fisher_score_sensor(df, sensor, start_cycles=50, end_cycles=50):
    begin_life = df[df['time, in cycles'] <= start_cycles][sensor]
    end_life = df[df['time, in cycles'] >= (df['time, in cycles'].max() - end_cycles + 1)][sensor]
    mean_diff = abs(begin_life.mean() - end_life.mean())
    within_var = begin_life.var() + end_life.var()
    return mean_diff / within_var

# Apply Fisher score calculation across each sensor
sensor_columns = [col for col in train.columns if col.startswith('sensor')]
fisher_scores = {sensor: fisher_score_sensor(train, sensor) for sensor in sensor_columns}

# Select the top sensors based on Fisher scores
top_sensors = sorted(fisher_scores, key=fisher_scores.get, reverse=True)[:5]
print("Top sensors selected based on Fisher score:", top_sensors)

# Calculate RUL for each engine
train['RUL'] = train.groupby('unit number')['time, in cycles'].transform(lambda x: x.max() - x)

# Apply EMA for each top sensor
ema_span = 50
for sensor in top_sensors:
    train[f'{sensor}_EMA'] = train.groupby('unit number')[sensor].transform(lambda x: x.ewm(span=ema_span, adjust=False).mean())
train = train[['unit number', 'time, in cycles'] + [f'{sensor}_EMA' for sensor in top_sensors] + ['RUL']]
import matplotlib.pyplot as plt
import os
# Add gradient columns to check for consecutive direction
for sensor in top_sensors:
    # Calculate the gradient between consecutive EMA points
    train[f'{sensor}_EMA_gradient'] = train.groupby('unit number')[f'{sensor}_EMA'].diff()

# Define function to check if EMA direction is consistent over 5 cycles
def check_consistent_direction(df, sensor, window=5):
    gradients = df[f'{sensor}_EMA_gradient']
    direction = np.sign(gradients)
    # Check if within a rolling window of 5, all directions are the same (either all 1 or all -1)
    return direction.rolling(window=window).apply(lambda x: all(x == x[0]), raw=True).fillna(0).astype(bool)

# Apply this function for each sensor and create a column indicating consistent direction over 5 cycles
for sensor in top_sensors:
    train[f'{sensor}_EMA_consistent_direction'] = train.groupby('unit number').apply(
        lambda x: check_consistent_direction(x, sensor)
    ).reset_index(level=0, drop=True)

In [ ]:
train.head()

In [ ]:
train.to_csv('train_FD004_EMA_Gradient.csv', index=False)

In [ ]:
    import os
    import matplotlib.pyplot as plt

    # Check for 5 consecutive cycles with consistent direction, where at least 4 sensors are TRUE simultaneously for each engine
    consecutive_cycles = 4
    required_true_sensors = 3  # Minimum number of sensors required to have consecutive TRUE values simultaneously

    output_dir = 'plots'
    os.makedirs(output_dir, exist_ok=True)

    # List to collect results for each engine
    results = []

    for engine_id, group in train.groupby('unit number'):
        # Create a DataFrame of consistent directions for each sensor
        consistent_directions = group[[f'{sensor}_EMA_consistent_direction' for sensor in top_sensors]]
        
        # Identify rows where at least 4 sensors are TRUE simultaneously
        sufficient_sensors_true = (consistent_directions.sum(axis=1) >= required_true_sensors)
        
        # Find rolling windows where this condition is TRUE for all 5 consecutive cycles
        consecutive_true = sufficient_sensors_true.rolling(window=consecutive_cycles).apply(lambda x: all(x), raw=True).fillna(0).astype(bool)
        
        # Get the index of the first cycle where the condition is met
        valid_cycles = group.loc[consecutive_true].index.tolist()
        Degradation_Onset = group.loc[valid_cycles[0], 'time, in cycles'] if valid_cycles else None
        
        # Print the first instance if it exists
        if Degradation_Onset:
            print(f"Engine {engine_id} meets the condition first at cycle {Degradation_Onset}.")
        
        # Plotting
        fig, axs = plt.subplots(2, 3, figsize=(15, 10))
        fig.suptitle(f'EMA Trend for Engine {engine_id}', fontsize=16)
        
        for i, sensor in enumerate(top_sensors):
            ax = axs[i // 3, i % 3]
            ema = group[f'{sensor}_EMA']
            ax.plot(group['time, in cycles'], ema, 'o--',label='EMA', color='green')
            
            # Mark the first cycle that meets the condition
            if Degradation_Onset:
                ax.axvline(x=Degradation_Onset, color='red', linestyle='--', label='Degradation Onset')
            
            ax.set_title(sensor)
            ax.set_xlabel('Cycles')
            ax.set_ylabel('EMA')
            ax.legend(loc='upper right')
        
        # Remove the last subplot if it’s not used
        fig.delaxes(axs[1, 2])
        
        plt.tight_layout(rect=[0, 0, 1, 0.96])
        plt.savefig(f"{output_dir}/engine_{engine_id}_ema.png", dpi=300)
        plt.show()
        plt.close(fig)

        # Add results to the list
        results.append({'unit number': engine_id, 'Degradation Onset': Degradation_Onset})

    # Convert results to DataFrame and save to CSV once
    results_df = pd.DataFrame(results)
    # Save the results to a CSV file with consecutive_cycles in the filename
    results_df.to_csv(f'results_{consecutive_cycles}_cycles.csv', index=False)

In [ ]:
from scipy.stats import ks_2samp

# Define a function to compare distribution similarity using KS test
def ks_test_distribution_similarity(df, sensor, degradation_onset):
    # Split data into before and after Degradation_Onset
    before_degradation = df[df['time, in cycles'] < degradation_onset][f'{sensor}_EMA']
    after_degradation = df[df['time, in cycles'] >= degradation_onset][f'{sensor}_EMA']
    
    # Apply KS test
    if len(before_degradation) > 0 and len(after_degradation) > 0:
        statistic, p_value = ks_2samp(before_degradation, after_degradation)
    else:
        statistic, p_value = None, None  # Not enough data
    
    return statistic, p_value

# Dictionary to hold the KS test results for each sensor and each engine
ks_test_results = []

for engine_id, group in train.groupby('unit number'):
    degradation_onset = results_df.loc[results_df['unit number'] == engine_id, 'Degradation Onset'].values[0]
    
    # Skip engines without a valid degradation onset
    if pd.notnull(degradation_onset):
        engine_results = {'unit number': engine_id, 'Degradation Onset': degradation_onset}
        
        for sensor in top_sensors:
            statistic, p_value = ks_test_distribution_similarity(group, sensor, degradation_onset)
            engine_results[f'{sensor}_KS_statistic'] = statistic
            engine_results[f'{sensor}_p_value'] = p_value
        
        ks_test_results.append(engine_results)

# Convert KS test results to a DataFrame
ks_test_results_df = pd.DataFrame(ks_test_results)
ks_test_results_df.to_csv('ks_test_results.csv', index=False)

# Display summary of results
for sensor in top_sensors:
    significant_count = (ks_test_results_df[f'{sensor}_p_value'] < 0.05).sum()
    print(f"For {sensor}, {significant_count} engines have a significantly different distribution (p < 0.05) before and after Degradation_Onset.")

In [ ]:
# Identify the sensor with the highest Fisher score
highest_fisher_sensor = max(fisher_scores, key=fisher_scores.get)
print(f"Sensor with the highest Fisher score: {highest_fisher_sensor}")

# Create a DataFrame to store engines where the distribution is significantly different for the highest Fisher score sensor
significant_engines = []

for _, row in ks_test_results_df.iterrows():
    # Extract KS statistic and p-value for the sensor with the highest Fisher score
    ks_statistic = row[f'{highest_fisher_sensor}_KS_statistic']
    p_value = row[f'{highest_fisher_sensor}_p_value']
    engine_id = row['unit number']
    
    # Check if the distribution is significantly different (p-value < 0.05)
    if pd.notnull(p_value) and p_value < 0.05:
        significant_engines.append({
            'unit number': engine_id,
            'KS statistic': ks_statistic,
            'p-value': p_value,
            'Inference': 'Significantly Different' if p_value < 0.05 else 'Not Significantly Different'
        })

# Convert significant engines list to a DataFrame
significant_engines_df = pd.DataFrame(significant_engines)

# Save the significant results DataFrame to a CSV file
significant_engines_df.to_csv(f'significant_engines_{highest_fisher_sensor}.csv', index=False)

# Display summary
print(f"Engines with significant distribution difference for {highest_fisher_sensor} sensor:")
significant_engines_df

In [ ]:
train_fd004 = pd.read_csv('train_FD004.csv', low_memory=False)
train_fd004.columns

In [ ]:
train.columns

In [ ]:
# Plot the histogram of sensor 16 from the original training data
plt.figure(figsize=(10, 6))
plt.hist(train_fd004['sensor measurement 16'], bins=50, color='skyblue', edgecolor='black')
plt.title('Histogram of Sensor 16')
plt.xlabel('Sensor 16')
plt.ylabel('Frequency')
plt.grid(axis='y', alpha=0.75)
plt.savefig('sensor_16_histogram.png', dpi=300)
plt.show()

In [ ]:
# Plot the histogram of sensor 16 from the original training data
plt.figure(figsize=(10, 6))
plt.hist(train['sensor measurement 16_EMA'], bins=50, color='skyblue', edgecolor='black')
plt.title('Histogram of Sensor 16')
plt.xlabel('Sensor 16')
plt.ylabel('Frequency')
plt.grid(axis='y', alpha=0.75)
plt.savefig('sensor_16_histogram.png', dpi=300)
plt.show()

In [ ]:
# Plot time series data for engine 1 for sensor 16
engine1 = train_fd004[train_fd004['unit number'] == 1]
plt.figure(figsize=(10, 6))
plt.plot(engine1['time, in cycles'], engine1['sensor measurement 16'], 'o--', label='Sensor 16', color='skyblue')
plt.title('Sensor 16 Time Series for Engine 1')
plt.xlabel('Time, in cycles')
plt.ylabel('Sensor 16')
plt.legend()
plt.grid(axis='y', alpha=0.75)
plt.savefig('sensor_16_time_series.png', dpi=300)
plt.show()

In [ ]:
# Plot time series data for engine 1 for sensor 16
engine1 = train_fd004[train_fd004['unit number'] == 1]
plt.figure(figsize=(10, 6))
plt.plot(engine1['time, in cycles'], engine1['sensor measurement 16'], 'o--', label='Sensor 16', color='skyblue')
plt.title('Sensor 16 Time Series for Engine 1')
plt.xlabel('Time, in cycles')
plt.ylabel('Sensor 16')
plt.legend()
plt.grid(axis='y', alpha=0.75)
plt.savefig('sensor_16_time_series.png', dpi=300)
plt.show()

In [ ]:
# Plot time series data for engine 1 for sensor 16 ema
engine1 = train[train['unit number'] == 1]
plt.figure(figsize=(10, 6))
plt.plot(engine1['time, in cycles'], engine1['sensor measurement 16_EMA'], 'o--', label='Sensor 16 EMA', color='green')
plt.title('Sensor 16 EMA Time Series for Engine 1')
plt.xlabel('Time, in cycles')
plt.ylabel('Sensor 16 EMA')
plt.legend()
plt.grid(axis='y', alpha=0.75)
plt.savefig('sensor_16_ema_time_series.png', dpi=300)